# Functools Module #

+ Higher Order Functions ==> functions that act on or return other functions

In [2]:
import functools

## Caching ##
### Cache Method (Memoize) ##

+ functools.cache() ==> function cache that stores results + memorises function calls for faster execution in future calls
+ recursive function call with input n ==> makes n+1 recursive calls (no previously cached results) + caches results at each call on call stack
+ recursive function call with input k where (k < n) ==> just looks up result from cache
+ recursive function call with input l where (l > n) ==> makes l-n new recursive calls, but n calls are already cached

### LRU Cache (Least Recently Used) ###

+ functools.lru_cache() ==> same functionality as functools.cache()
    + maxsize argument ==> allows you to specify how many most recent results to save (if maxsize limit is reached, least recently used results evicted from cache)
    + typed argument ==> True; function arguments of different types will be cached seperately (eg, 3 and 3.0 treated as distinct calls with distinct results)
    + useful for when you want to reuse previously computed values
    + access underlying function with functionName.__wrapped__() --> returns original function object 
    + functionName.cache_info() --> measure effectiveness of cache

In [21]:
@functools.lru_cache(maxsize=10)
def fibonacci(n):
    if n < 2:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print([fibonacci(n) for n in range(15)])
fibonacci.cache_info()

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]


CacheInfo(hits=26, misses=15, maxsize=10, currsize=10)

In [20]:
@functools.lru_cache(maxsize=None)
def factorial(n):
    if n <= 1:
        return 1
    else:
        return n*factorial(n-1)

CacheInfo(hits=0, misses=0, maxsize=15, currsize=0)

## Total Ordering Method ##

+ functools.total_ordering() ==> decorates class that defines one or more rich comparison methods (< , > __ gt __ , <= __ le __, >= __ ge __) with remaining comparison methods (class must also define __ eq __ method)
    + Tradeoff: slower execution but faster development time

In [29]:
@functools.total_ordering
class Number:
    def __init__(self, num: int):
        self.value = num
    
    def __lt__(self, other):
        return self.value < other.value

    def __eq__(self, other):
        return self.value == other.value

newnum = Number(6)
secnum = Number(10)
print(newnum.__lt__(secnum))
print(newnum.__gt__(secnum))
print(newnum.__le__(secnum))
print(newnum.__ge__(secnum))
print(newnum.__eq__(secnum))

True
False
True
False
False


## Partial Functions ##

+ functools.partial() ==> returns a new partial object which when called behaves like func called with args and keywords
    + more arguments supplied to partial object call are appended to args
    + "freezes" portion of function's arguments/keywords ==> results in new object with simplified signature
    + useful for function calls with one constant value but other arguments are changing

    + Partial Objects ==> callable objects created by partial() with 3 attributes...
        + partial.func == callable object or function (calls to partial object forwarded to func with new arguments and keywords)
        + partial.args == leftmost positional arguments that are prepended to positional arguments provided to partial object call
        + partial.keywords = keyword arguments supplied when partial object is called

In [35]:
def add(x: int, y: int) -> int:
    return x + y

newPartialAdd = functools.partial(add, y=3)
print(newPartialAdd(x=2))

5


## Reduce Method ##

+ functools.reduce() ==> applies function of two arguments cumulatively to items of iterable from left to right ==> reduces iterable to single cumulative value
    + left argument, x = accumulated value
    + right argument, y = update value from iterable
    + initializer argument (optional) ==> placed before items of iterable in calculation (serves as a default when iterable is empty)
    + performs function on FIRST TWO elements of iterable to generate a partial result and repeats until 1 cumulative value remains

In [44]:
functools.reduce(lambda x,y: x+y, ["xy", "xxx", "ge", "ifwif"])
#1 xy + xxx = xyxxx
#2 xyxxx + ge = xyxxxge
#3 xyxxxge + ifwif = xyxxxgeifwif

'xyxxxgeifwif'

In [49]:
import operator

def factorialReduce(n):
    return functools.reduce(lambda x,y: operator.mul(x,y), range(1,n+1)) ## [1,2,3,4]

factorialReduce(4)
#1 [1,2,3,4] ==> 1*2 = 2
#2 [2,3,4] ==> 2*3 = 6
#3 [6,4] ==> 6*4 = 24
#4 [24] ==> STOP

24

## SingleDispatch Method (Function Overloading) ##

+ Generic Function ==> function composed of multiple functions implementing the same operation for different types
+ Single Dispatch ==> form of generic function where implementation is chosen based on type of single argument
+ @functools.singledispatch ==> decorator that transforms a function into a single dispatch generic function.
    + @dispatchfunction.register(type) ==> adds overloaded implementations to function + generic function dispatches on type of FIRST argument
    + EG: if first argument is INT and second argument is FLOAT ==> generic function will dispatch to INT registered method
+ function overloading ==> functions with same name but different parameters
+ Generic function called with type that is NOT registered ==> calls generic function

+ @functools.singledispatchmethod ==> decorator that transforms a method into a single-dispatch function

In [73]:
@functools.singledispatch
def genericAdd(x,y):
    print(f"Executing generic function with arguments {x} and {y} of unregistered types {type(x), type(y)}")

@genericAdd.register(str)
def demo(x,y):
    print(f"Adding strings {x} and {y}")
    return x+y

@genericAdd.register(int)
def demo(x,y):
    print(f"Adding int {x} and {y}")
    return (x+y) - (x*y)

@genericAdd.register(float)
def demo(x,y):
    print(f"Adding float {x} and {y}")
    return x+y

@genericAdd.register(list)
def demo(x,y):
    print(f"Adding length of {x} to length of {y}")
    return len(x) + len(y)

genericAdd(1.3, 1)

Adding float 1.3 and 1


2.3

## Wraps Method and Decorator Functions ##

+ decorator ==> extends functionality of function without modifying existing code
+ @functools.wraps ==> decorator that transforms wrapper function to return original function name and NOT wrapper function name when executed
    + EG: addValues.__ name __ is "addValues" with @wraps decorator and is "wrapperFunction" without @wraps decorator

+ Common Use Cases for Decorators: tracking execution time of function calls, logging

In [94]:
import time
import operator

In [97]:
def decoratorFunction(func):

    @functools.wraps(func)
    def wrapperFunction(*args, **kwargs):
        print(f"Executing function {func.__name__} with arguments {args}")
        t1 = time.time()
        currResult = func(*args, **kwargs)
        t2 = time.time() - t1
        print(f"Function {func.__name__} ran in: {t2} seconds")
        return currResult

    return wrapperFunction

@decoratorFunction ## same syntax as ==> addValues = decoratorFunction(addValues)
def addValues(x,y):
    return x + y

@decoratorFunction
def subValues(x,y):
    return x-y

@decoratorFunction
def factorialReduce(n):
    return functools.reduce(lambda x,y: operator.mul(x,y), range(1,n+1))

Executing function factorialReduce with arguments (4,)
Function factorialReduce ran in: 1.0013580322265625e-05 seconds


24